# Programs, Circuits, and Jobs

In [ ]:
# imports for quantum circuit description
from qat.lang.AQASM import *

# quantum program declaration
prog = Program()
# qbit allocation
qbits = prog.qalloc(2)
# gate application
H(qbits[0]) # a Hadamard gate on the first qubit
CNOT(qbits[0], qbits[1]) # a CNOT gate from qubit 0 to qubit 1
circuit = prog.to_circ() # circuit generation
job = circuit.to_job() # this is the object that will executed
job = circuit.to_job(qubits=[0, 1]) # this restricts the sampling to the first 2 qubits
job = circuit.to_job(qubits=[0, 1], nbshots=33) # this restricts the sampling to 33 shots
# circuits can be displayed like so
%qatdisplay circuit --svg

# Quantum gates

All gates can be inverted using `.dag` and controlled using `.ctrl()` or `.ctrl(k)`.

In [ ]:
prog = Program()
qbits = prog.qalloc(5)
X.ctrl()(qbits[0], qbits[1])
RZ(0.322).dag()(qbits[0])
H.ctrl(4)(qbits)
circuit = prog.to_circ()
%qatdisplay circuit --svg

The gates will will need are the following:

In [ ]:
prog = Program()
qbits = prog.qalloc(4)
# Hadamard gate (to create superpositions and interferences)
H(qbits[0])
# CNOT gate: a xor (or controlled NOT)
CNOT(qbits[0], qbits[1])
# CCNOT gate or X.ctrl(k) gates: a logical and between two or more qubits
CCNOT(qbits[:-1])
X.ctrl(3)(qbits)
# Z gate: this one flips the phase of a qubit iff its in |1> state
Z(qbits[1])
# One can use its contolled version to flip a state iff all its qubits are in |1>
Z.ctrl(3)(qbits)
circuit = prog.to_circ()
%qatdisplay circuit --svg

# QRoutine

Sometimes it is useful to declare subroutines/subcircuits. You can think of them as large quantum gates wrapping a quantum circuit.

These are declared using `QRoutine` objects that have almost the same interface as `Programs`.

QRoutines can also be controlled/inverted

In [ ]:
routine = QRoutine()
# input/output wires are declared via .new_wires
wires = routine.new_wires(5)
for wire in wires:
    H(wire)
    RZ(1.5)(wire)
    
%qatdisplay routine --svg

inv = routine.dag()
%qatdisplay inv --svg
controlled = routine.ctrl(2)
%qatdisplay controlled --svg

## Compute/uncompute

In reversible computing, tt happens quite a lot that you'll need to:
- compute something
- compute another thing
- uncompute the first thing

For instance, lets say we want to

# Execution

In [ ]:
# importing a simulator
from qat.qpus import get_default_qpu
qpu = get_default_qpu()
# submitting a job to the simulator
result = qpu.submit(job)
for sample in result:
    print(sample.state, sample.probability)
    print(sample.state.value) # this one is a bit nicer to work with
    print(list(map(int, sample.state.value[0]))) # this constructs a proper bit-array